# Converting raw ANT .cnt
* This file will convert raw ANT-Neuro .cnt EEG files to a format that can be read by MNE-Python.
* The goal 
    * To read .cnt data using libeep library
    * Convert to numpy nDArray if not already
    * Use MNE-Pythons RawArray method to read EEG data.
* Conversion will be done using python2 because libeep was compiled with this version.
* The arrays OR MNE RawArray objects will be stored with pickle and later read with an updated version of MNE for python3

In [55]:
import os
import time

import libeep
import read_antcnt
import mne
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [57]:
path = '/run/media/edrias/5212-DECF/raw_data/'

In [3]:
# method to get names of all .cnt and .trg files and put them in a dictionary with raw_data folder as key
def raw_data_file_names(path):
    
    raw_data_folders = os.listdir(path)
    raw_data_dict = {}
    for folder in raw_data_folders:
        files = os.listdir(os.path.join(path, folder))
        raw_data_dict[folder] = [x for x in files if '.cnt' in x or '.trg' in x]
    return raw_data_dict

In [4]:
raw_data_dict = raw_data_file_names(path)

In [5]:
# read raw data - takes path of .cnt file.
def read_raw_cnt(path):
    return libeep.read_cnt(path)
    

In [6]:
# get file path of each .cnt file
file_dict_keys = raw_data_dict.keys()
cnt_paths = []

for key in file_dict_keys:
    files = raw_data_dict[key]
    cnt_files = [x for x in files if '.cnt' in x]
    
    for file in cnt_files:
        folder_path = os.path.join(path, key)
        cnt_file = os.path.join(folder_path, file)
        cnt_paths.append(cnt_file)
        

In [7]:
# needed to get a sample size interval 
def get_interval(sample_size, interval, divide_by = 10):
    return (sample_size/divide_by) + (interval * (sample_size/divide_by))

In [8]:
# getting intervals samples of a single .cnt file
raw_cnt = read_raw_cnt(cnt_paths[1])
samples_size = raw_cnt.get_sample_count()
print "samples size: ", samples_size
intervals = [0]

for i in range(9):
    interval = get_interval(samples_size, i)
    intervals.append(interval)
intervals.append(samples_size)
print intervals

samples size:  1011555
[0, 101155, 202310, 303465, 404620, 505775, 606930, 708085, 809240, 910395, 1011555]


In [9]:
# read through each interval for a .cnt file and check for errors.
for i in range(1,len(intervals)):
    start = time.time()
    raw = raw_cnt.get_samples(intervals[i -1], intervals[i])
    end = time.time()
    elapsed = end - start
    print "shape: ", np.array(raw).shape
    print "elapsed time: ", elapsed
    print raw[:100]


shape:  (12947840,)
elapsed time:  3.06668400764
[-6966.18701171875, 1175.3416748046875, -22793.154296875, -6048.85693359375, -5745.017578125, -1031.1273193359375, -2750.224365234375, -1106.986083984375, -6661.94287109375, -2270.815673828125, -2463.35888671875, 11445.8076171875, 1326.1396484375, -2614.212158203125, -1035.062744140625, 13350.9375, -3876.3544921875, 2788.65966796875, -1196.6925048828125, 11992.8916015625, -2814.534423828125, -4314.68017578125, 12005.5986328125, 15816.171875, -1804.4267578125, -2534.601806640625, 3177.277099609375, 5049.8388671875, -1937.09228515625, -9618.7236328125, 18.794580459594727, -3072.61962890625, 14240.076171875, 1678.9886474609375, -10266.419921875, -2264.305419921875, 10639.9208984375, 3877.090087890625, -2592.401611328125, -5501.7548828125, 1492.6611328125, -8070.74560546875, -8627.134765625, -4168.00146484375, -7124.138671875, 0.0, 0.0, 0.0, -645.8935546875, -7176.513671875, -9370.8447265625, -7263.35107421875, -8848.5693359375, -8437.975585

shape:  (12947840,)
elapsed time:  2.61226320267
[-6677.5009765625, 1679.94482421875, -35687.48828125, -5489.6357421875, -5388.06787109375, -471.4276428222656, -2467.05517578125, -768.7939453125, -5662.4833984375, -2674.310546875, -3498.366455078125, 11376.421875, 2781.22998046875, -2092.70849609375, -1235.3482666015625, 13104.76953125, -3666.616455078125, 3186.47216796875, -1808.601318359375, 11756.4326171875, -3146.584228515625, -3368.864013671875, 12114.6142578125, 16307.4248046875, -1696.1649169921875, -1973.2838134765625, 3035.894775390625, 4038.99560546875, -1917.1207275390625, -10357.302734375, 1396.1871337890625, -3541.877197265625, 14530.8212890625, 1756.3001708984375, -10522.62890625, -2857.291015625, 10823.140625, 4513.49462890625, -2921.049072265625, -4944.9052734375, 1203.0921630859375, -8447.6298828125, -8072.60302734375, -4724.869140625, -5520.9169921875, 0.0, 0.0, 0.0, 1775.48095703125, -7290.255859375, -8725.9814453125, -6671.74462890625, -8677.2841796875, -7487.377929

In [21]:
# method to strip tuple of get_channels method of libeep
raw = np.array(raw)
def get_channels(raw_cnt):
    channels = []
    for i in range(raw_cnt.get_channel_count()):
        channels.append(raw_cnt.get_channel(i)[0])
    return channels

In [24]:
# get channels and create info
channels = get_channels(raw_cnt= raw_cnt)
info = mne.create_info(ch_names= channels, sfreq=512)

In [52]:
n_chan = raw_cnt.get_channel_count();
n_times = raw_cnt.get_sample_count()

data = np.empty((n_chan+1, intervals[1]), dtype=np.double)
from numpy import asarray
x = asarray(raw_cnt.get_samples(0,intervals[1]))
x.shape  = (intervals[1],n_chan) 
print(x.shape)
data[:-1] = x.transpose() # get into a shape that can be read by MNE


(101155, 128)


In [71]:
# check how many files can't be read by libeep.
for path in cnt_paths:
    try: 
        libeep.read_cnt(path)
    except:
        print "Error with cnt: ", path

Error with cnt:  /run/media/edrias/5212-DECF/raw_data/SA20140331/20140331_1556.cnt
